In [85]:
from bs4 import BeautifulSoup
from datetime import datetime
import feedparser
import json
import os
import pandas as pd
import requests
import tabula

In [86]:
url = 'https://www.regione.sicilia.it'
feed = feedparser.parse(url+'/feed')

In [87]:
f = [field for field in feed['entries'] if 'bollettino settimanale' in field['title']]
link = f[0]['links'][0]['href']
print(link)

http://www.regione.sicilia.it/la-regione-informa/covid-bollettino-settimanale-aumento-nuovi-casi-calo-prime-dosi


In [88]:
html = requests.get(link)
soup = BeautifulSoup(html.text, 'html.parser')
links = soup.find_all('a')

In [89]:
for link in links:
    if ('.pdf' in link.get('href')):
        pdf = url+link.get('href')

print(pdf)

https://www.regione.sicilia.it/sites/default/files/2021-10/Report%20Completo%2027%20Ottobre%202021.pdf


In [90]:
r = requests.get(pdf, stream=True)
with open('./downloads/'+datetime.now().strftime("%Y%m%d")+'.pdf', 'wb') as f:
    f.write(r.content)
    print("PDF Scaricato.")

PDF Scaricato.


In [91]:
files = []
path = './downloads/'

for file in os.listdir(path):
    if (file.endswith('.pdf')):
        files.append(file)

files.sort()
file = files[0]

## Tabula

In [92]:
print('Leggo PDF...attendi...')
df = tabula.read_pdf_with_template(path+file, 'template_vaccini.json', pandas_options={'header': None}, multiple_tables=False, silent=True)
print('Ho letto.')

Leggo PDF...attendi...
Ho letto.


In [93]:
join = pd.concat(df).reset_index(drop=True)
join.columns = ['Comune', 'Vaccinati', 'Immunizzati']

In [94]:
join

,Comune,Vaccinati,Immunizzati
0,Agrigento,"87,37%","84,36%"
1,Alessandria della Rocca,"84,65%","82,72%"
2,Aragona,"88,61%","86,08%"
3,Bivona,"85,46%","83,37%"
4,Burgio,"89,60%","88,12%"
...,...,...,...
385,San Vito Lo Capo,"74,59%","68,89%"
386,Santa Ninfa,"87,36%","84,00%"
387,Trapani,"79,17%","74,29%"
388,Valderice,"80,79%","76,27%"


In [95]:
print('Converto in CSV.')
join.to_csv('tabula-'+file[:-4]+'.csv', index=None, header=True)

Converto in CSV.
